# Phase 3v2: QLoRA 파인튜닝 (SOS/EOS + CoT 학습)

## 📌 목적
Phase 2v2에서 생성한 `training_dataset_v2.jsonl`을 사용하여
Unsloth 기반 QLoRA 파인튜닝을 수행합니다.

### 기존 대비 개선점
- ✅ SOS/EOS 토큰 포함 → 생성 종료 시점 학습
- ✅ CoT(Thinking) 포함 → 추론 과정 학습
- ✅ 노이즈 제거된 고품질 데이터
- ✅ RPM 기반 피처/팩터 정보 포함

## ⚙️ 사전 조건
- `phase2_dataset_v2.ipynb` 실행으로 `outputs/training_dataset_v2.jsonl` 존재
- Colab T4 이상 GPU 할당

## 📂 출력 파일
```
models/
├── llama3_popular_post_lora_v2/          # LoRA 어댑터
└── llama3_popular_post_lora_v2/merged_16bit/  # 16bit 병합 모델
```

---
## 0. 필수 패키지 설치

In [ ]:
%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install -q --no-deps "xformers<0.0.27" "trl<0.9.0" "peft<0.10.0" "accelerate<1.0.0" "bitsandbytes<0.43.0"
%pip install -q datasets

print("✅ Unsloth & 관련 패키지 설치 완료")

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")

---
## 1. 경로 및 데이터 로드

In [ ]:
from pathlib import Path
import json
from datasets import Dataset

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
MODEL_DIR    = PROJECT_ROOT / "models"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_JSONL = OUTPUT_DIR / "training_dataset_v2.jsonl"
if not TRAIN_JSONL.exists():
    raise FileNotFoundError(f"{TRAIN_JSONL}가 없습니다. Phase 2v2를 먼저 실행하세요.")

training_data = []
with open(TRAIN_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            training_data.append(json.loads(line))

print(f"✅ 데이터셋 로드 완료: {len(training_data):,}개 레코드")

---
## 2. Unsloth 모델 로드 + QLoRA 설정

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print(f"✅ Unsloth 모델 로드 완료")
print(f"  BOS token: {tokenizer.bos_token} (id: {tokenizer.bos_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (id: {tokenizer.eos_token_id})")

---
## 3. 데이터셋 템플릿 적용 (SOS/EOS 토큰 확인)

In [ ]:
# ── text 필드의 SOS/EOS를 실제 토크나이저 토큰으로 대체 ──────
BOS = tokenizer.bos_token  # <|begin_of_text|>
EOS = tokenizer.eos_token  # <|end_of_text|>

def fix_special_tokens(text: str) -> str:
    """placeholder를 실제 토크나이저 토큰으로 대체.
    Llama-3에서는 placeholder와 실제 토큰이 동일하므로
    이 함수는 안전장치 역할을 합니다.
    다른 모델 사용 시 여기서 매핑을 변경하세요."""
    text = text.replace("<|begin_of_text|>", BOS)
    text = text.replace("<|end_of_text|>", EOS)
    return text

for record in training_data:
    if "text" in record:
        record["text"] = fix_special_tokens(record["text"])

dataset = Dataset.from_list(training_data)

# 검증: 첫 번째 샘플 확인
print("✅ 데이터셋 준비 완료")
print(f"   레코드 수: {len(dataset):,}")
print(f"\n📝 첫 번째 샘플 (앞 300자):")
print(dataset[0]["text"][:300])
print(f"\n📝 마지막 50자:")
print(dataset[0]["text"][-50:])

# 토큰화 검증: BOS/EOS가 올바르게 포함되는지 확인
sample_ids = tokenizer(dataset[0]["text"], return_tensors="pt").input_ids[0]
print(f"\n🔍 토큰화 검증:")
print(f"  첫 토큰 ID: {sample_ids[0].item()} (BOS={tokenizer.bos_token_id})")
print(f"  끝 토큰 ID: {sample_ids[-1].item()} (EOS={tokenizer.eos_token_id})")
print(f"  BOS 포함: {sample_ids[0].item() == tokenizer.bos_token_id}")
print(f"  EOS 포함: {sample_ids[-1].item() == tokenizer.eos_token_id}")

---
## 4. SFTTrainer 설정 및 학습

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# ── 패딩 설정 (EOS 학습 보장) ─────────────────────────────
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=str(OUTPUT_DIR / "finetune_logs_v2"),
        save_strategy="epoch",
        save_total_limit=3,
    ),
)

print("✅ Trainer 설정 완료. 학습을 시작합니다...")
trainer.train()
print("✅ 학습 완료")

---
## 5. 모델 저장 (LoRA + 16bit 병합)

In [ ]:
OUTPUT_MODEL_DIR = MODEL_DIR / "llama3_popular_post_lora_v2"
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# LoRA 어댑터 저장
model.save_pretrained(str(OUTPUT_MODEL_DIR))
tokenizer.save_pretrained(str(OUTPUT_MODEL_DIR))
print(f"✅ LoRA 어댑터 저장 완료: {OUTPUT_MODEL_DIR}")

# 16bit 병합 모델 저장
MERGED_DIR = OUTPUT_MODEL_DIR / "merged_16bit"
FastLanguageModel.for_inference(model)
model.save_pretrained_merged(
    str(MERGED_DIR),
    tokenizer,
    save_method="merged_16bit",
)
print(f"✅ 16bit 병합 모델 저장 완료: {MERGED_DIR}")

---
## 6. 다음 단계
→ `phase4_generation_v2.ipynb`에서 CoT 기반 인기글 생성 수행